In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

import sys
sys.path.append(os.path.join(git_path,"figures"))
import plotting_utils

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

hems = {s:[] for s in subjs}
for s in subjs:
    pt = img_pipe.freeCoG(f"{s}_complete",hem='stereo',subj_dir=data_path)
    elecs = pt.get_elecs()['elecmatrix']
    if sum(elecs[:,0] > 0) >= 1:
        hems[s].append('rh')
    if sum(elecs[:,0] < 0) >= 1:
        hems[s].append('lh')

color_palette = pd.read_csv(os.path.join(git_path,"figures","color_palette.csv"))
spkr_color = color_palette.loc[color_palette['color_id']=='perception']['hex'].values[0]
mic_color = color_palette.loc[color_palette['color_id']=='production']['hex'].values[0]

### epoch data

In [ ]:
si_tmin = 0; si_tmax = 1; erp_tmin = -.5; erp_tmax = 2; reject = None; baseline = None
epochs = dict(); ch_names = dict()
for s in tqdm(subjs):
    epochs[s] = dict(); spkr_epochs, mic_epochs = [], []
    for b in blocks[s]:
        blockid = f'{s}_{b}'
        raw_fpath = os.path.join(data_path,f"sub-{s}",s,blockid,"HilbAA_70to150_8band",
                                 "ecog_hilbAA70to150.fif")
        raw = mne.io.read_raw_fif(raw_fpath,preload=True,verbose=False)
        ch_names[s] = raw.info['ch_names']
        fs = raw.info['sfreq']
        # Spkr events
        eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                 f"{blockid}_spkr_sn_all.txt")
        with open(eventfile,'r') as f:
            c = csv.reader(f,delimiter='\t')
            events = np.array([[int(float(row[0])*fs),int(float(row[1])*fs),int(row[2])] for row in c])
        spkr_epochs.append(mne.Epochs(raw,events,tmin=erp_tmin,tmax=erp_tmax,
                                      baseline=baseline,reject=reject,verbose=False))
        # Mic events
        eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                 f"{blockid}_mic_sn_all.txt")
        with open(eventfile,'r') as f:
            c = csv.reader(f,delimiter='\t')
            events = np.array([[int(float(row[0])*fs),int(float(row[1])*fs),int(row[2])] for row in c])
        mic_epochs.append(mne.Epochs(raw,events,tmin=erp_tmin,tmax=erp_tmax,
                                      baseline=baseline,reject=reject,verbose=False))
    epochs[s]['spkr'] = mne.concatenate_epochs(spkr_epochs)
    epochs[s]['mic'] = mne.concatenate_epochs(mic_epochs)

### Calculate the suppression index
Suppression index ($SI$) is formalized as:

$SI_{n} = \frac{1}{t}\sum\limits_{t=0}^{t=1}{H_\gamma L_{n,t}}-{H_\gamma S_{n,t}}$ , 

where $H\gamma$ is the averaged high gamma activity averaged across a time window of interest for either perception ($L$) or production ($S$). Here we will calculate $SI$ in a 0-1000ms window relative to sentence onset.

In [ ]:
x = epochs[s]['mic'].times
si_inds = [np.where(x==si_tmin)[0][0],np.where(x==si_tmax)[0][0]]
si = dict()
for s in subjs:
    subj_si = []
    for i,ch in enumerate(epochs[s]['mic'].info['ch_names']):
        spkr_resp = epochs[s]['spkr'].get_data(picks=[ch]).squeeze()[:,si_inds[0]:si_inds[1]].mean(0).mean(0)
        mic_resp = epochs[s]['mic'].get_data(picks=[ch]).squeeze()[:,si_inds[0]:si_inds[1]].mean(0).mean(0)
        subj_si.append(spkr_resp-mic_resp)
    si[s] = np.array(subj_si)
# Normalize between 0 and 1
si_min = np.hstack((list(si.values()))).min()
si_max = np.hstack((list(si.values()))).max()
for s in subjs:
    si[s] = ((si[s]-si_min)/(si_max-si_min)) * 2 - 1

In [ ]:
# Create colormap
# Shift SI values so that the max value is 1 without normalizing
# Just for cmap! We still report native cmap values
# Max val for spkr - we shift so this is 1
max_val = np.hstack((list(si.values()))).max(); min_val = np.hstack((list(si.values()))).min()
# Apply the shift
normed_si = dict()
for s in subjs:
    subj_si = []
    for ch in si[s]:
        subj_si.append((ch-min_val)/(max_val-min_val))
    # Normalize
    normed_si[s] = np.array(subj_si)
elec_colors_rgb = dict()
elec_colors_hex = dict()
cmap = LinearSegmentedColormap.from_list('my_gradient', (
    # Edit this gradient at https://eltos.github.io/gradient/#0:954997-20:954997-45:FFFFFF-50:FFFFFF-55:FFFFFF-80:117632-100:117632
    (0.000, (0.584, 0.286, 0.592)),
    (0.200, (0.584, 0.286, 0.592)),
    (0.450, (1.000, 1.000, 1.000)),
    (0.500, (1.000, 1.000, 1.000)),
    (0.550, (1.000, 1.000, 1.000)),
    (0.800, (0.067, 0.463, 0.196)),
    (1.000, (0.067, 0.463, 0.196))))
for s in subjs:
    elec_colors_rgb[s] = [cmap(f) for f in normed_si[s]]
    elec_colors_hex[s] = np.array(
        [plotting_utils.rgb_to_hex(int(ec[0]*255),int(ec[1]*255),int(ec[2]*255)) for ec in elec_colors_rgb[s]])

### write colormap values to dataframe

In [ ]:
# Save to pandas
df = pd.DataFrame(columns=['subj','hem','ch_name','x','y','z','r','g','b','a'])
incl_df = pd.read_csv(os.path.join(git_path,"analysis","included_electrodes_pvals_only.csv"))

for hem in ['lh','rh']:
    for s in [ss for s in subjs if hem in hems[ss]]:
        blockid = "_".join([s,blocks[s][0]])
        fif_ch_names = mne.io.read_raw_fif(os.path.join(data_path,f"sub-{s}",blockid,"HilbAA_70to150_8band",
            "ecog_hilbAA70to150.fif"), preload=False, verbose=False).info['ch_names']
        pt = img_pipe.freeCoG(f'{s}_complete',hem=hem, subj_dir=ip)
        e, a = imaging_utils.clip_4mm_elecs(pt,hem=hem,elecfile_prefix="TDT_elecs_all_warped")
        e, a = imaging_utils.clip_outside_brain_elecs(pt,elecmatrix=e,anatomy=a,hem=hem,
                                                      elecfile_prefix="TDT_elecs_all_warped")
        fs_ch_names = [aa[0][0] for aa in a]
        incl_ch_names = incl_df.loc[incl_df['subject']==s]['channel'].values
        for ch in incl_ch_names:
            if ch.replace('-','') in [c.replace('-','') for c in fif_ch_names]:
                fif_idx = [c.replace('-','') for c in fif_ch_names].index(ch.replace('-',''))
                r,g,b,a = elec_colors_rgb[s][fif_idx]
                elecfile_idx = [c.replace('-','') for c in fs_ch_names].index(ch.replace('-',''))
                x,y,z = e[elecfile_idx,:]
                new_row = pd.DataFrame({'subj':[s],'hem':[hem],'ch_name':[ch],'x':[x],'y':[y],'z':[z],
                                       'r':[r],'g':[g],'b':[b],'a':[a]})
                df = df.append(new_row, ignore_index=True)
df.to_csv(os.path.join(git_path,"figures","figure_1","csv","figure_1_cmap.csv"),index=False)